In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

In [2]:
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [3]:
import os
import sys

if os.path.abspath('../../..') not in sys.path:
    sys.path.append(os.path.abspath('../../..'))

In [4]:
from gensim.models.coherencemodel import CoherenceModel

from src.corpus.documents import load_documents
from src.preprocessing.documents import filter_documents, preprocess_documents
from src.models import AuthorTopicModel
from src.models.callbacks import ModelCheckpoint
from src.corpus import keywords
from src.preprocessing.keywords import create_eta
from src.plotting.topic_models import TopicModelVisualizer

/Users/yasas/Documents/Projects/climate-matters/evo-lyzer/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [5]:
docs = filter_documents(docs=load_documents(verbose=True), verbose=True)

Extracting Unique Documents: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 670323/670323 [00:01<00:00, 422546.98it/s]


85590 duplicates removed from a total of 670323 documents.


Selecting Documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 82860/82860 [00:00<00:00, 1637416.57it/s]


In [6]:
print('Number of Total Documents: {}'.format(len(docs)))

Number of Total Documents: 82860


In [7]:
if os.path.exists('../../../models/v0-T6-E1/model.pt'):
    model = AuthorTopicModel.load('../../../models/v0-T6-E1/model.pt')
else:
    model_checkpoint = ModelCheckpoint('../../../models', version=0)
    model = AuthorTopicModel(
        num_topics=6, 
        passes=10,
        iterations=20,
        keywords=keywords,
        callbacks=[model_checkpoint],
        verbose=True,
    )
    model.fit(docs)

Training the Model (Epoch 10): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 82860/82860 [01:43<00:00, 802.58it/s]


In [8]:
preprocessed_docs = model.preprocess(docs)

Extracting Author Docs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 82860/82860 [00:00<00:00, 1079383.90it/s]


In [9]:
model.get_coherence(preprocessed_docs)

-3.578021862708173

In [11]:
topics_proba = model.transform(preprocessed_docs)

Inferring the Document Topic Probabilities: 100%|███████████████████████████████████████████████████████████████████████████████████| 82860/82860 [00:34<00:00, 2405.97it/s]


In [12]:
topics_proba.shape

(82860, 6)

In [14]:
topic_viz = TopicModelVisualizer(model, verbose=True)

topic_viz.fit(preprocessed_docs)

In [31]:
component = topic_viz.get_html_component()

/Users/yasas/Documents/Projects/climate-matters/evo-lyzer/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
/Users/yasas/Documents/Projects/climate-matters/evo-lyzer/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/yasas/Documents/Projects/climate-matters/evo-lyzer/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/yasas/Documents/Projects/climate-matters/evo-lyzer/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the 

In [37]:
# component.display('html')